# COS360 - Otimização 2020.2
## Bernardo Maiorano e Igor Amaral

$$
\begin{align}
\displaystyle \min \ \  &f(x)\\
\text{sujeito a:}\  \ \ &x \in \Omega\\
\\
f(x_1,x_2,x_3,x_4) &= -30x_1 - 10x_2 - 40x_3 - 12x_4\\
\Omega = \{x \in \mathbb{R}^4 :& \  {33x_1 + 14x_2 + 47x_3 + 11x_4 \le 59\}} \\
\end{align}
$$

##  Estudo da função

In [1]:
import numpy as np
import pandas as pd

In [2]:
coefficients = np.array([-30, -10, -40, -12])
omega = np.array([33, 14, 47, 11])

def f(x):
    return x @ coefficients

def G(x):
    # Restrição de desigualdade
    return omega @ x <= 59

# Gradiente constante
grad_f = coefficients

Temos uma função linear no R4, com apenas uma restrição igualmente linear. Dessa forma, temos uma função convexa, já que a mesma é linear (um plano no R4). Isso garante que, caso haja um mínimo local contido no espaço do domínio, esse mínimo também será global.  Temos também que, pelo fato da função ser linear, teremos um gradiente constante Grad_f = (-30,-10,-40,-12). Além disso, a Hessiana dessa função é uma matriz 4x4 vazia.  Pelo fato desta ser uma função linear, podemos resolver a mesma pelo Simplex. Dessa forma, com a restrição dada e sabendo que no simplex x1, x2, x3 e x4 >= 0, temos o seguinte valor ótimo:

In [3]:
# mínimo pelo simplex: x = [0,0,0,59/11]
f([0,0,0,59/11])

-64.36363636363636

### Penalidade Exterior

Pelo fato desta função ser restrita, para poder utilizar os métodos de programação não-linear, é necessário transformar o problema em um problema irrestrito. Faremos isso a seguir.

Para transformar nosso problema restrito em irrestrito. Temos que usar o método de penalidade externa ou interna.

Assim, o problema transformado seria: $\min \phi(x,\rho) = f(x) + \rho P(x)$ onde $\rho$ é o parâmetro de penalidade e $P(x)$ é a função de penalidade.

Vamos usar a função de penalidade externa: $\displaystyle P(x) = \sum_{h \in H} h(x)^2 + \sum_{g \in G} \max[0, g(x)]^2$

Para o novo problema temos o gradiente: $\nabla \varPhi(x,\rho) = \nabla f(x) + \rho\nabla P(x) = \begin{cases}
    \nabla f(x) &\text{se}_\ g(x) \le 0\\
    \nabla f(x) + \rho\nabla P(x) &\text{se}_\ g(x) > 0 \\
\end{cases}$

No nosso caso, a função objetivo torna-se: $ \\ $
$$
\phi(x,\rho) = {-30x_1-10x_2-40x_3-12x_4} + \rho \max[0,{33x_1+14x_2+47x_3+11x_4-59}]^2
$$

In [4]:
rho = 10000000

def P_Exterior(x, p = rho):
    return f(x) + p*pow(max(0, omega@x-59),2)

$$
\nabla \varPhi(x,\rho) = \begin{cases} 
\begin{pmatrix}
-30 + 66\rho({33x_1+14x_2+47x_3+11x_4-59}) \\
-10 + 28\rho({33x_1+14x_2+47x_3+11x_4-59}) \\
-40 + 94\rho({33x_1+14x_2+47x_3+11x_4-59}) \\
-12 + 22\rho({33x_1+14x_2+47x_3+11x_4-59}) \\
\end{pmatrix} &\text{se}_\ g(x) \le 0 \\
\qquad\qquad\begin{pmatrix}-30 \ \ -10 \ \ -40 \ \ -12 \end{pmatrix}^T &\text{se}_\ g(x) > 0 \\
\end{cases}
$$

In [5]:
def grad_Pext(x, p= rho):
    if G(x): return grad_f
    return np.array([(grad_f[i] + (p*2*omega[i]*(x@omega-59))) for i in range(4)])

$$
\nabla^2 \varPhi(x,\rho) = \begin{cases} 
\begin{pmatrix}
2178\rho&  924\rho& 3102\rho& 726\rho \\
924\rho& 392\rho& 1316\rho& 308\rho \\
3102\rho& 1316\rho& 4418\rho& 1034\rho \\
726\rho& 308\rho& 1034\rho& 242\rho \\
\end{pmatrix} &\text{se}_\ g(x) \le 0 \\
\quad\qquad\begin{pmatrix}
0 \quad 0\quad 0\quad 0 \\
0 \quad 0\quad 0\quad 0 \\
0 \quad 0\quad 0\quad 0 \\
0 \quad 0\quad 0\quad 0
\end{pmatrix} &\text{se}_\ g(x) > 0 \\
\end{cases}
$$

In [6]:
# L4 = 3L1 => det = 0 => matriz não inversivel
def hessian_Pext(x, p= rho):
    hessian = np.zeros((4,4))
    if G(x): return hessian
    for i in range(4):
        hessian[i] = [(p*2*omega[i]*omega[j]) for j in range(4)]
    return hessian

z = np.array([0,0,0,50],  dtype=np.float64)
hessian = hessian_Pext(z, p=1)
print(hessian)
print("\nL4*3 = {} = L1".format(hessian[3]*3))

[[2178.  924. 3102.  726.]
 [ 924.  392. 1316.  308.]
 [3102. 1316. 4418. 1034.]
 [ 726.  308. 1034.  242.]]

L4*3 = [2178.  924. 3102.  726.] = L1


Pelo fato da Hessiana não ser uma matriz inversível, não podemos aplicar o Método de Newton para essa função.

Além disso, para o Método Quase-Newton, teríamos que fazer a diferença dos gradientes, porém para a função dada, conforme explicado anteriormente, o gradiente é constante, portando sua diferença é sempre zero. Isso também nos impede de usar o Método Quase-Newton. Portanto usaremos apenas o Método do Gradiente.

## Implementação

In [7]:
def armijo(x, direction, f, Grad):
    t = 1
    gamma = 0.8
    n = 0.25
    n_iter = 0
    
    while (f(x + t*direction) > f(x) + n*t*Grad(x)@direction):
        t = t*gamma
        n_iter += 1
        
    # print("Armijo:",t,n_iter)
    return t, n_iter

In [8]:
def stop(x, prev_x, n_iter, max_iters = 50):
    return (np.allclose(x, prev_x, atol=1e-16) or n_iter > max_iters)# and G(x)
 
def Gradiente(x, f, Grad):

    n_iter = 0
    prev_x = float('inf')*np.ones(4)
    while not stop(x,prev_x,n_iter):
        direction = -Grad(x)
        t, calls_armijo = armijo(x,direction, f, Grad)
        prev_x = x
        x = x + t*direction
        n_iter += 1
    
    return x, f(x), n_iter, calls_armijo

In [9]:
def quase_Newton(x,f,Grad):

    n_iter = 0
    prev_x = float('inf')*np.ones(4)
    H = np.identity(4,dtype=np.float64)
    while not stop(x,prev_x,n_iter):
        direction = -(H@Grad(x))
        t,calls_armijo = armijo(x,direction, f, Grad)
        prev_x = x
        x = x + t*direction
        n_iter += 1
        # atualizar H (definida positiva)
        p = x - prev_x
        q = Grad(x) - Grad(prev_x)
#         part1 = np.outer(p,p)/(p@q)
#         part2 = np.dot(np.outer(np.dot(H,p),p),H)
#         part3 = p@H@p
#         H = H + part1 -(part2/part3)
        
    return x, f(x), n_iter, calls_armijo

In [10]:
x = np.array([0,0,0,0], dtype=np.float64)
Gradiente(x, P_Exterior, grad_Pext)

(array([0.56333545, 0.18777848, 0.75111393, 0.22533418]),
 -51.526415524699985,
 6,
 86)

In [11]:
x = np.array([0,0,0,0], dtype=np.float64)
quase_Newton(x, P_Exterior, grad_Pext)

(array([0.56333545, 0.18777848, 0.75111393, 0.22533418]),
 -51.526415524699985,
 6,
 86)

In [12]:
n = 10
results = []
while n:
    x0 = np.random.rand(4)
    opt_point, opt_value, n_iter, calls_armijo = Gradiente(x0, P_Exterior, grad_Pext)
    if G(opt_point):
        results.append([x0,n_iter,calls_armijo,opt_point, opt_value])
        n -= 1
        
df = pd.DataFrame(results, columns = ['X0','Iter.','Call.  Armijo','Opt.  Point','Opt.  Value'])
df

,X0,Iter.,Call. Armijo,Opt. Point,Opt. Value
0,"[0.45128560606516377, 0.5998656233690712, 0.79...",7,84,"[0.4194848676231054, 0.5832822000156911, 0.748...",-50.331496
1,"[0.37451482936033664, 0.703039395088067, 0.216...",6,81,"[0.5386713562821185, 0.7577582373953278, 0.435...",-52.223258
2,"[0.5869956394709213, 0.5648204048307541, 0.493...",4,77,"[0.5832523868826627, 0.5628683778132787, 0.487...",-52.399009
3,"[0.5406587061246403, 0.42804168830580525, 0.03...",5,77,"[0.8596240128121747, 0.5343634572016499, 0.461...",-51.185759
4,"[0.4824665480013476, 0.5763155618806649, 0.336...",7,75,"[0.6535268679963379, 0.6333356685456617, 0.564...",-50.738893
5,"[0.17031470604700838, 0.25357346730053576, 0.0...",7,71,"[0.5886336052732067, 0.3930131003759353, 0.619...",-51.773323
6,"[0.9288909344665226, 0.17472430261376426, 0.39...",6,74,"[0.9734711045911986, 0.18958435932198955, 0.45...",-52.407999
7,"[0.6728622121844583, 0.30457499834884116, 0.00...",7,75,"[0.9449744075687242, 0.39527906347692987, 0.36...",-52.459404
8,"[0.10533622767665929, 0.5621526054550053, 0.66...",6,75,"[0.2345302919156383, 0.6052172935346651, 0.836...",-50.333231
9,"[0.21334574943340423, 0.35219589851235, 0.0682...",6,77,"[0.5648226079774594, 0.4693548513603684, 0.536...",-52.447944


In [13]:
x = np.array([0,0,0,500], dtype=np.float64)
print(G(x))
opt_point, opt_value, n_iter, calls_armijo = Gradiente(x, P_Exterior, grad_Pext)
print(G(opt_point),opt_point,opt_value)

False
True [-47.43989681 -24.73892233 -72.17881914 487.56949843] -1293.0950877044731


In [14]:
x = np.array([-100,-100,-100,5000], dtype=np.float64)
print(G(x))
opt_point, opt_value, n_iter, calls_armijo = Gradiente(x, P_Exterior, grad_Pext)
print(G(opt_point),opt_point,opt_value)

False
True [-497.07034766 -307.0639715  -704.13431915 4895.95713283] -12603.362683130868
